# Importing Necessary Libraries and Modules

In this first cell, we import all the required libraries and modules for the project. 
This includes the following:
- **`crewai` framework**: For defining agents, tasks, and tools in our robotic system.
- **`requests` library**: To handle REST API calls.
- **`pydantic`**: For data validation.
- **`dotenv`**: To load environment variables securely.
- **`warnings`**: To suppress warnings for a cleaner output during video recording.

These libraries will enable us to program, execute, and control the robotic servos.


In [7]:
from crewai import Agent, Task, Crew
from crewai.tools import BaseTool
import requests
from time import sleep
from pydantic import Field
from dotenv import load_dotenv
import os

# Warning control
import warnings
warnings.filterwarnings('ignore')


# Defining the Servo Control Tool

In this cell, we define the `ServoControlTool`, a custom tool designed to control servo motors using REST services hosted on an ESP32 device. Here's what the tool can do:
- **Move individual servos**: Specify the servo and the target angle, ensuring it's within the allowed range.
- **Perform predefined actions**: Execute actions like "blink" or "double blink".
- **Execute complex sequences**: Handle lists of commands, including servo movements, actions, and pauses.

The tool includes:
1. **Validation of servo names and angles**: Ensures commands won't cause errors or unsafe movements.
2. **Error handling**: Manages network issues and invalid inputs gracefully.
3. **Support for synchronous and asynchronous execution**: Adapts to various use cases.

The servo ranges are predefined, and the tool interacts with the ESP32 via HTTP requests.


In [8]:
ESP32_URL = "http://192.168.1.141/"
class ServoControlTool(BaseTool):
    name: str = "servo_control_tool"
    description: str = """
                    This tool controls servo motors via HTTP requests and can execute predefined actions.
                    Commands should be provided as a list, where each command is a dictionary. The available keys and their meanings are as follows:
                    
                    type: Specifies the type of command. It can be one of the following:
                      - servo: To move a specific servo to a given angle.
                      - action: To perform a predefined action, such as 'blink' or 'double blink'.
                      - pause: To insert a delay between commands.
                    
                    For type = 'servo':
                      - servo_name: The name of the servo to control (e.g., top_left, bottom_left, Xarm for horizontal movement, Yarm for vertical movement).
                      - angle: The angle to move the servo to, within its allowed range.
                    
                    For type = 'action':
                      - action: The action to perform (e.g., blink, double blink).
                    
                    For type = 'pause':
                      - duration: The duration of the pause in seconds.
                    
                    Example of Commands:
                    - Move the upper left eyelid to 60°:
                      type: servo, servo_name: top_left, angle: 60
                    - Perform a blink:
                      type: action, action: blink
                    - Pause for 0.5 seconds:
                      type: pause, duration: 0.5
                    
                    Ensure that all commands are valid and that angles are within the allowed range for each servo.
                    If a command is invalid, an error will be returned.
                    """
    base_url: str = Field(default=ESP32_URL, description="Base URL of the ESP32.")
    
    servo_ranges: dict = Field(
        default={
            "top_left": (55, 100),
            "bottom_left": (55, 100),
            "top_right": (55, 105),
            "bottom_right": (65, 120),
            "Xarm": (40, 130),
            "Yarm": (60, 115),
        },
        description="Dictionary defining servo names and their movement ranges.",
    )

    def move_servo(self, servo_name: str, angle: int) -> str:
        """
        Move a servo to the specified angle.

        Args:
            servo_name (str): Name of the servo to move.
            angle (int): Target angle for the servo.

        Returns:
            str: Status message about the result of the operation.
        """
        if servo_name not in self.servo_ranges:
            return f"Error: Servo '{servo_name}' not recognized."

        min_angle, max_angle = self.servo_ranges[servo_name]
        if angle < min_angle or angle > max_angle:
            return (
                f"Error: Angle {angle} is out of range for servo '{servo_name}' "
                f"(allowed range: {min_angle}-{max_angle})."
            )

        try:
            response = requests.get(f"{self.base_url}mover", params={"servo": servo_name, "angle": angle})
            if response.status_code == 200:
                return f"Servo '{servo_name}' successfully moved to {angle} degrees."
            else:
                return f"Error moving servo '{servo_name}': HTTP {response.status_code}."
        except requests.exceptions.RequestException as e:
            return f"Connection error while moving servo '{servo_name}': {e}"

    def perform_action(self, action: str) -> str:
        """
        Perform a predefined action like 'blink' or 'double blink'.

        Args:
            action (str): Name of the action to perform.

        Returns:
            str: Status message about the result of the operation.
        """
        valid_actions = ["blink", "double blink"]
        if action not in valid_actions:
            return f"Error: Action '{action}' not recognized. Valid actions: {valid_actions}."

        # Direct endpoint for actions
        try:
            response = requests.get(f"{self.base_url}{action.replace(' ', '_')}")
            if response.status_code == 200:
                return f"Action '{action}' successfully performed."
            else:
                return f"Error performing action '{action}': HTTP {response.status_code}."
        except requests.exceptions.RequestException as e:
            return f"Connection error while performing action '{action}': {e}"

    def execute_commands(self, commands: list) -> str:
        """
        Execute a list of commands, including servo movements, actions, and pauses.

        Args:
            commands (list): List of commands. Each command is a dictionary with keys:
                - "type": "servo", "action", or "pause"
                - Additional keys depending on the type

        Returns:
            str: Combined status messages of all executed commands.
        """
        results = []
        for command in commands:
            if command["type"] == "servo":
                servo_name = command.get("servo_name")
                angle = command.get("angle")
                if servo_name and angle is not None:
                    results.append(self.move_servo(servo_name, angle))
                else:
                    results.append("Error: Missing 'servo_name' or 'angle' in servo command.")

            elif command["type"] == "action":
                action = command.get("action")
                if action:
                    results.append(self.perform_action(action))
                else:
                    results.append("Error: Missing 'action' in action command.")

            elif command["type"] == "pause":
                duration = command.get("duration")
                if duration is not None:
                    sleep(duration)
                    results.append(f"Paused for {duration} seconds.")
                else:
                    results.append("Error: Missing 'duration' in pause command.")

            else:
                results.append(f"Error: Unknown command type '{command['type']}'.")
            sleep(0.1)

        return "\n".join(results)

    def _run(self, commands: list = None) -> str:
        """
        Synchronously execute a list of commands.

        Args:
            commands (list): List of commands to execute.

        Returns:
            str: Status message about the result of the operation.
        """
        if commands:
            return self.execute_commands(commands)
        else:
            return "Error: No commands provided."

    async def _arun(self, commands: list = None) -> str:
        """
        Asynchronously execute a list of commands.

        Args:
            commands (list): List of commands to execute.

        Returns:
            str: Status message about the result of the operation.
        """
        return self._run(commands)


# Configuring Environment Variables for OpenAI API

In this cell, we:
1. **Load environment variables**: Using `load_dotenv()` to securely retrieve sensitive information like the API key from a `.env` file.
2. **Set the API key**: Retrieve the `OPENAI_API_KEY` and set it as an environment variable for authentication.
3. **Configure the OpenAI model**: Specify the model to use, such as `gpt-4o` or `gpt-3.5-turbo`. This configuration determines the AI's capabilities during execution.

> **Note**: Using `.env` files ensures sensitive information isn't hardcoded, improving security and flexibility.


In [9]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# Set API key and model to use gpt-4o
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o"  # Update to gpt-4o
#os.environ["OPENAI_MODEL_NAME"] = "gpt-3.5-turbo"



# Defining Agents and Tasks for Eyelid Movement

In this cell, we define:
1. **Agents**: Each servo is represented as an agent with:
   - A **role**: Specifies the functional role of the servo (e.g., "Upper Left Eyelid").
   - A **goal**: Describes the agent's purpose (e.g., "Move to convey the desired expression").
   - A **backstory**: Provides technical specifications, limitations, and the servo's functional role in the system.

2. **Tasks**: Each task assigns a specific action to an agent:
   - A **description**: Explains the task's purpose, such as moving the eyelid to match a given expression.
   - An **expected output**: Defines what should be confirmed after the task execution.
   - A **tool**: Specifies the `ServoControlTool` to execute the task.
   - An **agent**: Links the task to the relevant servo.

> **Example**:
- The `servo_top_left` agent controls the upper left eyelid, with a range of 55° (closed) to 100° (open).
- The `task_move_top_left_eyelid` task instructs this servo to move based on the desired expression.


In [10]:
servo_top_left = Agent(
    role="Upper Left Eyelid",
    goal="Determine the necessary angles for this servo and move it to convey the desired expression.",
    backstory="""
        Technical specs: 
        - Brand: Tower Pro 
        - Model: Micro Servo 9g SG90
        - Range: 0° to 180°

         Limitations:
        - Range in this system: 55° to 100° 
        
        Functional role in the system:
        - This is a servo that controls the upper left eyelid of the robot's left eye.
        - 55° = (Completely closed)
        - 100° = (Completely open)

    """
)

servo_bottom_left = Agent(
    role="Lower Left Eyelid",
    goal="Determine the necessary angles for this servo and move it to convey the desired expression.",
    backstory="""
        Technical specs:
        - Brand: Tower Pro 
        - Model: Micro Servo 9g SG90
        - Range: 0° to 180°

        Limitations:
        - Range in this system: 55° to 100° 

        Functional role in the system:
        - This is a servo that controls the lower left eyelid of the robot's left eye.
        - 55°  = (Completely open)
        - 100° = (Completely closed)


    """,
    verbose=True
)

servo_top_right = Agent(
    role="Upper Right Eyelid",
    goal="Determine the necessary angles for this servo and move it to convey the desired expression.",
    backstory="""
        Technical specs: 
        - Brand: Tower Pro 
        - Model: Micro Servo 9g SG90
        - Range: 0° to 180°

        Limitations:
        - Range in this system: 55° to 105° 
        
        Functional role in the system:
        - This is a servo that controls the upper right eyelid of the robot's right eye.
        - 55° = (Completely opened)
        - 105° = (Completely closed)
    """,
    verbose=True
)

servo_bottom_right = Agent(
    role="Lower Right Eyelid",
    goal="Determine the necessary angles for this servo and move it to convey the desired expression.",
    backstory="""
        Technical specs:
        - Brand: Tower Pro 
        - Model: Micro Servo 9g SG90
        - Range: 0° to 180°°

        Limitations:
        - Range in this system: 65° to 120° 

        Functional role in the system:
        - This is a servo that controls the lower right eyelid of the robot's right eye.
        - 65°  = (Completely closed)
        - 120° = (Completely opened)
    """,
    verbose=True
)

servo_xarm = Agent(
    role="Horizontal eye movement",
    goal="Determine the necessary angle for horizontal movement of the eyes and move the servo.",
    backstory="""
        Technical specs:
        - Brand: Tower Pro 
        - Model: Micro Servo 9g SG90
        - Range: 0° to 180°

        Limitations:
        - Range in this system: 40° to 130°
        
        Functional role in the system:
        - This is a servo that controls the horizontal movement of both eyes on a robot face.
        - 40°  = (Completely left)
        - 130° = (Completely right)
    """
)

servo_yarm = Agent(
    role="Vertical eye movement",
    goal="Determine the necessary angle for vertical movement of the eyes and move the servo.",
    backstory="""
        Technical specs:
        - Brand: Tower Pro 
        - Model: Micro Servo 9g SG90
        - Range: 0° to 180°

        Limitations:
        - Range in this system: 60° to 115°
        
        Functional role in the system:
        - This is a servo that controls the vertical movement of both eyes on a robot face.
        - 60°  = (Completely down)
        - 115° = (Completely up)

    """
)
# If you had more hardware components  you would continue defining 
# each one as an independent Agent with its own range and backstory.

# --------------------------------------------------------------------
# Task Definition
# --------------------------------------------------------------------

task_move_top_left_eyelid = Task(
    description="""
        Move the upper left eyelid to the correct angle for the desired expression: {expression}.
    """,
    expected_output="Confirm that the upper left eyelid has moved to the computed angle.",
    tools=[ServoControlTool()],
    agent=servo_top_left
)

task_move_bottom_left_eyelid = Task(
    description="""
        Move the lower left eyelid to the correct angle for the desired expression: {expression}.
    """,
    expected_output="Confirm that the lower left eyelid has been moved to the correct angle.",
    tools=[ServoControlTool()],
    agent=servo_bottom_left
)

task_move_top_right_eyelid = Task(
    description="""
        Move the upper right eyelid to the correct angle for the desired expression: {expression}.
    """,
    expected_output="Confirm that the upper right eyelid has moved to the computed angle.",
    tools=[ServoControlTool()],
    agent=servo_top_right
)

task_move_bottom_right_eyelid = Task(
    description="""
        Move the lower right eyelid to the correct angle for the desired expression: {expression}.
    """,
    expected_output="Confirm that the lower right eyelid has been moved to the correct angle.",
    tools=[ServoControlTool()],
    agent=servo_bottom_right
)

task_xarm = Task(
    description="""
        Move the eyes horizontally to the specified angle for the expression: {expression}.
    """,
    expected_output="Confirm that the horizontal movement of the eyes has been executed.",
    tools=[ServoControlTool()],
    agent=servo_xarm
)

task_yarm = Task(
    description="""
        Move the eyes vertically to the specified angle for the expression: {expression}.
    """,
    expected_output="Confirm that the vertical movement of the eyes has been executed.",
    tools=[ServoControlTool()],
    agent=servo_yarm
)



# Defining the Crew for Multi-Agent Orchestration

In this cell, we define the `Crew`, which acts as the orchestrator for the system. The crew coordinates the agents and their assigned tasks, ensuring seamless collaboration. Here's how it's structured:
1. **Agents**: Includes the servos responsible for moving the eyelids (`servo_top_left` and `servo_bottom_left`).
2. **Tasks**: Specifies the actions these agents need to perform (e.g., move the eyelids to match a given expression).
3. **Additional parameters**:
   - `verbose`: Enables detailed logs during execution, useful for debugging or demonstrations.
   - `memory`: Determines whether the system maintains a memory of past actions or operates statelessly.

This setup allows for efficient multi-agent orchestration, making it easy to add more agents or tasks as needed.


In [12]:
crew = Crew(
    agents=[
        servo_top_left,
        servo_bottom_left,
        servo_top_right,
        servo_bottom_right,
        servo_xarm,
        servo_yarm

    ],
    tasks=[
        task_move_top_left_eyelid,
        task_move_bottom_left_eyelid,
        task_move_top_right_eyelid,
        task_move_bottom_right_eyelid,
        task_xarm,
        task_yarm
 
    ],
    verbose=True,
    memory=False  # Depends on your needs
)

Overriding of current TracerProvider is not allowed


# Running the Multi-Agent System

In this cell, we execute the crew's orchestration by providing input data:
1. **Inputs**: A dictionary containing the expression we want the robot to convey (e.g., "very surprised" or "asleep").
2. **Kickoff**: The `kickoff` method triggers the crew's execution, sending the input to the relevant agents and tasks.
3. **Result**: The result of the execution is printed, showing how the agents processed the input and the outcome of their tasks.

> This step demonstrates the system in action, transforming a high-level input (like an expression) into coordinated servo movements.


In [13]:
inputs = {
    "expression": "suspicious, looking to the right",  # Example expression: 'asleep', 'surprised', etc.
}

# Start the crew orchestration with the specified inputsfr
result = crew.kickoff(inputs=inputs)

# Display the result of the execution
print(result)


# Agent: Upper Left Eyelid
## Task: 
        Move the upper left eyelid to the correct angle for the desired expression: suspicious, looking to the right.
    


# Agent: Upper Left Eyelid
## Thought: To convey a "suspicious" expression while looking to the right, the upper left eyelid should be slightly lowered, suggesting a partial closure, but not completely. A suitable angle might be slightly above 55° (completely closed) to achieve this effect. Given the range from 55° to 100°, setting it to around 70° should provide a slightly lowered eyelid, enhancing the suspicious look.
## Using tool: servo_control_tool
## Tool Input: 
"{\"commands\": [{\"type\": \"servo\", \"servo_name\": \"top_left\", \"angle\": 70}]}"
## Tool Output: 
Servo 'top_left' successfully moved to 70 degrees.


# Agent: Upper Left Eyelid
## Final Answer: 
The upper left eyelid has been moved to 70 degrees to convey a suspicious expression.


# Agent: Lower Left Eyelid
## Task: 
        Move the lower left eyelid to